<a href="https://colab.research.google.com/github/HeywantPark/py_notebook/blob/main/face_trans_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import os

# PyTorch 확장 모듈 캐시 디렉토리 경로
cache_dir = os.path.expanduser('~/.cache/torch_extensions')
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

# 필요한 패키지 설치
!pip install faiss-cpu wget gdown

# DualStyleGAN 리포지토리 클론
!git clone https://github.com/williamyang1991/DualStyleGAN.git

# 클론한 리포지토리로 디렉토리 변경
os.chdir('DualStyleGAN')


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 68.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=4a15d17be19fa539fe22a1bfb0dd190766c6c7a64374975a869b118f356d5f3b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Cloning into 'DualStyleGAN'...
remote: Enumerating objects: 760, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 760 (delta 192), reused 171 (delta 169), pack-reused 529
Receiving objects: 100% (760/760), 26.24 MiB | 50.61 MiB/s, done.
Resolving deltas: 100% (378/378), done.


In [ ]:
import torch
import numpy as np
from argparse import Namespace
from torchvision import transforms
import gdown

# CUDA가 사용 가능한지 확인하고, 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 디렉토리 설정
MODEL_DIR = os.path.join(os.getcwd(), 'checkpoint')
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# 지원되는 스타일 타입 목록
style_types = ['cartoon', 'caricature', 'anime', 'arcane', 'comic', 'pixar', 'slamdunk']

# 모델을 다운로드할 URL 정보
MODEL_PATHS = {
    "encoder": {"id": "1NgI4mPkboYvYw3MWcdUaQhkr0OWgs9ej", "name": "encoder.pt"},
    "cartoon-G": {"id": "1exS9cSFkg8J4keKPmq2zYQYfJYC5FkwL", "name": "generator.pt"},
    "cartoon-N": {"id": "1JSCdO0hx8Z5mi5Q5hI9HMFhLQKykFX5N", "name": "sampler.pt"},
    "cartoon-S": {"id": "1ce9v69JyW_Dtf7NhbOkfpH77bS_RK0vB", "name": "refined_exstyle_code.npy"},
    "pixar-G": {"id": "1TgH7WojxiJXQfnCroSRYc7BgxvYH9i81", "name": "generator.pt"},
    "pixar-N": {"id": "18e5AoQ8js4iuck7VgI3hM_caCX5lXlH_", "name": "sampler.pt"},
    "pixar-S": {"id": "1I9mRTX2QnadSDDJIYM_ntyLrXjZoN7L-", "name": "exstyle_code.npy"},
    "anime-G": {"id": "1BToWH-9kEZIx2r5yFkbjoMw0642usI6y", "name": "generator.pt"},
    "anime-N": {"id": "19rLqx_s_SUdiROGnF_C6_uOiINiNZ7g2", "name": "sampler.pt"},
    "anime-S": {"id": "17-f7KtrgaQcnZysAftPogeBwz5nOWYuM", "name": "refined_exstyle_code.npy"},
}

# Google Drive에서 파일을 다운로드하는 함수
def download_file(file_id, file_name):
    url = f"https://drive.google.com/uc?id={file_id}"
    output = os.path.join(MODEL_DIR, file_name)

    if os.path.exists(output):
        os.remove(output)  # 기존 파일 삭제

    gdown.download(url, output, quiet=False)
    return output


In [ ]:
# 필요한 패키지 설치
!pip install ninja gdown

# 필요한 모델 모듈을 가져옴
from model.dualstylegan import DualStyleGAN
from model.sampler.icp import ICPTrainer
from model.encoder.psp import pSp

# 사용자로부터 스타일 타입을 선택받는 함수
def select_style_type():
    print("사용 가능한 스타일 타입:")
    for i, style in enumerate(style_types):
        print(f"{i}: {style}")
    style_idx = int(input("원하는 스타일 타입의 번호를 입력하세요: "))
    if 0 <= style_idx < len(style_types):
        return style_types[style_idx]
    else:
        print("유효하지 않은 입력입니다. 기본 스타일로 픽사를 선택합니다.")
        return 'pixar'

# 모델 다운로드를 위한 함수 정의
def download_file(file_id, file_name):
    url = f"https://drive.google.com/uc?id={file_id}"
    output = os.path.join(MODEL_DIR, file_name)

    # 해당 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(output), exist_ok=True)

    if not os.path.exists(output):
        gdown.download(url, output, quiet=False)
    return output

# 선택한 스타일 타입에 맞게 모델을 다운로드하고 로드하는 함수
def load_models(style_type):
    print(f"선택된 스타일 타입: {style_type}")

    encoder_path = download_file(MODEL_PATHS["encoder"]["id"], MODEL_PATHS["encoder"]["name"])
    generator_path = download_file(MODEL_PATHS[style_type + "-G"]["id"], os.path.join(style_type, MODEL_PATHS[style_type + "-G"]["name"]))
    sampler_path = download_file(MODEL_PATHS[style_type + "-N"]["id"], os.path.join(style_type, MODEL_PATHS[style_type + "-N"]["name"]))
    style_code_path = download_file(MODEL_PATHS[style_type + "-S"]["id"], os.path.join(style_type, MODEL_PATHS[style_type + "-S"]["name"]))

    # DualStyleGAN 생성기 로드
    generator = DualStyleGAN(1024, 512, 8, 2, res_index=6)
    generator.eval()
    ckpt = torch.load(generator_path, map_location=device)
    generator.load_state_dict(ckpt["g_ema"])
    generator = generator.to(device)

    # pSp 인코더 로드
    ckpt = torch.load(encoder_path, map_location=device)
    opts = ckpt['opts']
    opts['checkpoint_path'] = encoder_path
    opts = Namespace(**opts)
    opts.device = device
    encoder = pSp(opts)
    encoder.eval()
    encoder = encoder.to(device)

    # Extrinsic style code 로드
    exstyles = np.load(style_code_path, allow_pickle=True).item()

    # 샘플러 네트워크 로드
    icptc = ICPTrainer(np.empty([0, 512*11]), 128)
    icpts = ICPTrainer(np.empty([0, 512*7]), 128)
    ckpt = torch.load(sampler_path, map_location=device)
    icptc.icp.netT.load_state_dict(ckpt['color'])
    icpts.icp.netT.load_state_dict(ckpt['structure'])
    icptc.icp.netT = icptc.icp.netT.to(device)
    icpts.icp.netT = icpts.icp.netT.to(device)

    print(f'Style type "{style_type}"에 대한 모델이 성공적으로 로드되었습니다!')
    return generator, encoder, exstyles, icptc, icpts

# 스타일 타입 목록
style_types = ['cartoon', 'caricature', 'anime', 'arcane', 'comic', 'pixar', 'slamdunk']

# 모델 경로 설정
MODEL_DIR = '/content/DualStyleGAN/checkpoint'
MODEL_PATHS = {
    "encoder": {"id": "1NgI4mPkboYvYw3MWcdUaQhkr0OWgs9ej", "name": "encoder.pt"},
    "cartoon-G": {"id": "1exS9cSFkg8J4keKPmq2zYQYfJYC5FkwL", "name": "generator.pt"},
    "cartoon-N": {"id": "1JSCdO0hx8Z5mi5Q5hI9HMFhLQKykFX5N", "name": "sampler.pt"},
    "cartoon-S": {"id": "1ce9v69JyW_Dtf7NhbOkfpH77bS_RK0vB", "name": "refined_exstyle_code.npy"},
    "anime-G": {"id": "1BToWH-9kEZIx2r5yFkbjoMw0642usI6y", "name": "generator.pt"},
    "anime-N": {"id": "19rLqx_s_SUdiROGnF_C6_uOiINiNZ7g2", "name": "sampler.pt"},
    "anime-S": {"id": "17-f7KtrgaQcnZysAftPogeBwz5nOWYuM", "name": "refined_exstyle_code.npy"},
    # 추가 스타일 경로 설정 가능
}

# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 스타일 타입 선택 및 모델 로드
style_type = select_style_type()
generator, encoder, exstyles, icptc, icpts = load_models(style_type)



사용 가능한 스타일 타입:
0: cartoon
1: caricature
2: anime
3: arcane
4: comic
5: pixar
6: slamdunk
원하는 스타일 타입의 번호를 입력하세요: 0
선택된 스타일 타입: cartoon


Downloading...
From (original): https://drive.google.com/uc?id=1exS9cSFkg8J4keKPmq2zYQYfJYC5FkwL
From (redirected): https://drive.google.com/uc?id=1exS9cSFkg8J4keKPmq2zYQYfJYC5FkwL&confirm=t&uuid=466eb88a-e852-4753-ab7d-15f0a90b2b9d
To: /content/DualStyleGAN/checkpoint/cartoon/generator.pt
100%|██████████| 308M/308M [00:03<00:00, 96.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JSCdO0hx8Z5mi5Q5hI9HMFhLQKykFX5N
To: /content/DualStyleGAN/checkpoint/cartoon/sampler.pt
100%|██████████| 5.00M/5.00M [00:00<00:00, 90.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ce9v69JyW_Dtf7NhbOkfpH77bS_RK0vB
To: /content/DualStyleGAN/checkpoint/cartoon/refined_exstyle_code.npy
100%|██████████| 11.7M/11.7M [00:00<00:00, 61.7MB/s]


Loading pSp from checkpoint: /content/DualStyleGAN/checkpoint/encoder.pt
Style type "cartoon"에 대한 모델이 성공적으로 로드되었습니다!


In [ ]:
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from model.encoder.align_all_parallel import align_face

from google.colab import drive
drive.mount('/content/drive')

# 데이터 디렉토리 설정
DATA_DIR = os.path.join(os.getcwd(), 'data')

# 이미지 전처리 및 정렬을 위한 변환 정의
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

# 얼굴 정렬 함수 정의
def run_alignment(image_path):
    # dlib의 얼굴 랜드마크 모델 로드
    modelname = '/content/shape_predictor_68_face_landmarks.dat'
    if not os.path.exists(modelname):
        import wget, bz2
        wget.download('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2', modelname+'.bz2')
        with bz2.BZ2File(modelname+'.bz2') as zipfile:
            data = zipfile.read()
            with open(modelname, 'wb') as f:
                f.write(data)
    predictor = dlib.shape_predictor(modelname)

    # 얼굴 정렬 수행
    try:
        aligned_image = align_face(filepath=image_path, predictor=predictor)
        return aligned_image
    except Exception as e:
        print(f"얼굴 정렬 실패: {e}")
        return None

# 이미지 시각화 함수 정의
def visualize(tensor):
    image = tensor.clone().detach().cpu().numpy()
    image = (image * 0.5 + 0.5) * 255.0
    image = image.clip(0, 255).astype('uint8').transpose(1, 2, 0)
    plt.imshow(image)
    plt.axis('off')

# 입력 이미지 경로 설정
image_path = "/content/drive/MyDrive/data/hyewon_test.jpeg"  # 사용하려는 이미지 경로

# 얼굴 정렬 여부 설정
if_align_face = True

# 얼굴 정렬 및 변환 수행
if if_align_face:
    aligned_image = run_alignment(image_path)
    if aligned_image is None:
        raise RuntimeError("얼굴 정렬에 실패하여 프로그램을 종료합니다.")
    if aligned_image.mode != 'RGB':
        aligned_image = aligned_image.convert('RGB')  # 3채널로 변환
    I = transform(aligned_image).unsqueeze(dim=0).to(device)
else:
    I = transform(Image.open(image_path).convert("RGB")).unsqueeze(dim=0).to(device)

# 정렬된 이미지 시각화
plt.figure(figsize=(10, 10), dpi=30)
visualize(I[0].cpu())
plt.show()

# 스타일 ID 및 키 설정
style_id = 0  # 스타일 ID 선택 (0부터 시작)
stylename = list(exstyles.keys())[style_id]

# 캐릭터 이미지 생성
with torch.no_grad():
    img_rec, instyle = encoder(I, randomize_noise=False, return_latents=True,
                               z_plus_latent=True, return_z_plus_latent=True, resize=False)
    img_rec = torch.clamp(img_rec.detach(), -1, 1)

    latent = torch.tensor(exstyles[stylename]).repeat(2, 1, 1).to(device)
    # latent[0] for both color and structure transfer, latent[1] for only structure transfer
    latent[1, 7:18] = instyle[0, 7:18]
    exstyle = generator.generator.style(latent.reshape(latent.shape[0] * latent.shape[1], latent.shape[2])).reshape(latent.shape)

    img_gen, _ = generator([instyle.repeat(2, 1, 1)], exstyle, z_plus_latent=True,
                           truncation=0.7, truncation_latent=0, use_res=True, interp_weights=[0.6]*7+[1]*11)
    img_gen = torch.clamp(img_gen.detach(), -1, 1)

    # 색상 레이어 비활성화
    img_gen2, _ = generator([instyle], exstyle[0:1], z_plus_latent=True,
                            truncation=0.7, truncation_latent=0, use_res=True, interp_weights=[0.6]*7+[0]*11)
    img_gen2 = torch.clamp(img_gen2.detach(), -1, 1)

# 파일 이름 중복을 피하기 위한 함수 정의
def get_unique_filename(base_path):
    count = 1
    base, ext = os.path.splitext(base_path)
    new_path = base_path

    while os.path.exists(new_path):
        new_path = f"{base}({count}){ext}"
        count += 1

    return new_path

# 이미지 저장 함수 정의
def save_image(tensor, base_path):
    unique_path = get_unique_filename(base_path)
    image = tensor.clone().detach().cpu().numpy()
    image = (image * 0.5 + 0.5) * 255.0
    image = image.clip(0, 255).astype('uint8').transpose(1, 2, 0)
    Image.fromarray(image).save(unique_path)
    return unique_path

# 저장 및 출력할 이미지 경로 설정
base_path_gen = "/content/drive/MyDrive/generated_image.png"
base_path_gen2 = "/content/drive/MyDrive/generated_image_no_color.png"

# 이미지 저장 및 경로 출력
output_path_gen = save_image(img_gen[0], base_path_gen)
output_path_gen2 = save_image(img_gen2[0], base_path_gen2)

# 이미지가 제대로 저장되었는지 확인
print(f"이미지가 다음 경로에 저장되었습니다: {output_path_gen}, 크기: {os.path.getsize(output_path_gen)} bytes")
print(f"색상 레이어 비활성화된 이미지가 다음 경로에 저장되었습니다: {output_path_gen2}, 크기: {os.path.getsize(output_path_gen2)} bytes")

# 생성된 캐릭터 이미지 시각화
try:
    plt.figure(figsize=(10, 10), dpi=30)
    plt.imshow(Image.open(output_path_gen))
    plt.axis('off')
    plt.show()
    plt.close()  # figure를 닫아 메모리 해제
except Exception as e:
    print(f"이미지를 불러오는 중 오류 발생: {e}")

# 색상 레이어 비활성화된 이미지 시각화
try:
    plt.figure(figsize=(10, 10), dpi=30)
    plt.imshow(Image.open(output_path_gen2))
    plt.axis('off')
    plt.show()
    plt.close()  # figure를 닫아 메모리 해제
except Exception as e:
    print(f"이미지를 불러오는 중 오류 발생: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
이미지가 다음 경로에 저장되었습니다: /content/drive/MyDrive/generated_image(13).png, 크기: 1363168 bytes
색상 레이어 비활성화된 이미지가 다음 경로에 저장되었습니다: /content/drive/MyDrive/generated_image_no_color(12).png, 크기: 1132398 bytes
